In [ ]:
import os
from utils import parse_folder
from sklearn.svm import OneClassSVM
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
import numpy as np
import matplotlib.pyplot as plt
import cv2

%matplotlib inline

In [ ]:
import cPickle as pickle

In [ ]:
from joblib import Parallel, delayed

In [ ]:
in_folder = '/media/shared/dr/DiabeticRetinopathy/train_orig/'
names = np.asarray(parse_folder(in_folder, "jpeg"))

In [ ]:
num_samples = 1000
resize_size = (25, 25)
train_split = 0.9

In [ ]:
idx = np.arange(names.shape[0])
rng = np.random.RandomState(seed=12345)
rng.shuffle(idx)
X_names = names[idx[:num_samples]]

In [ ]:
def load_img(fname):
    img = cv2.imread(fname, 0)
    im = cv2.resize(img, resize_size).ravel().astype(np.float32)
#     im[im==0] = im[im!=0].mean()
    im = im / 255.
    return im

In [ ]:
# a = load_img(X_names[0])
# plt.imshow(a.reshape(resize_size))
# plt.colorbar()

In [ ]:
with Parallel(n_jobs=-1) as parallel:
    X_unnorm = parallel(delayed(load_img)(fname)
                 for fname in list(X_names))
    X_unnorm = np.asarray(list(X_unnorm), dtype=np.float32)

In [ ]:
train_samp = int(train_split * num_samples)
X_unnorm_train = X_unnorm[:train_samp]
X_unnorm_val = X_unnorm[train_samp + 1:]

In [ ]:
# X_mean = X_unnorm_train.mean(axis=0)
# X_std = X_unnorm_train.std(axis=0)
# X_train = (X_unnorm_train - X_mean) / X_std
# X_val = (X_unnorm_val - X_mean) / X_std
X_train = X_unnorm_train
X_val = X_unnorm_val

In [ ]:
# a = load_img(X_names[1])
# plt.imshow(X_train[1].reshape(resize_size))

In [ ]:
param_grid = {'nu': [0.005, 0.01, 0.05, .1],
              'gamma': [0.001, 0.005, 0.01, 0.05, 0.1], }

for nu in param_grid['nu']:
    for gamma in param_grid['gamma']:
        clf = OneClassSVM(nu=nu, kernel="rbf", gamma=gamma)
        clf.fit(X_train)
        print "##########"
        print "nu: %f and gamma: %f" % (nu, gamma)
        print "Error training: %d/%d" % (X_train[clf.predict(X_train)==-1].shape[0], X_train.shape[0])
        print "Error training: %d/%d" % (X_val[clf.predict(X_val)==-1].shape[0], X_val.shape[0])

In [ ]:
# One class SVM
clf = OneClassSVM(nu=0.01, kernel="rbf", gamma=0.01)
clf.fit(X_train)

In [ ]:
print "Error training: %d/%d" % (X_train[clf.predict(X_train)==-1].shape[0], X_train.shape[0])
print "Error training: %d/%d" % (X_val[clf.predict(X_val)==-1].shape[0], X_val.shape[0])

In [ ]:
pickle.dump(clf, open('retinal_img_clf.pkl', 'wb'))

In [ ]:
clf2 = pickle.load(open('full_retinal_img_clf.pkl', 'rb'))

In [ ]:
print "Error training: %d/%d" % (X_train[clf2.predict(X_train)==-1].shape[0], X_train.shape[0])
print "Error training: %d/%d" % (X_val[clf2.predict(X_val)==-1].shape[0], X_val.shape[0])